In [2]:
import os
import csv
import requests
#from vertexai.generative_models import GenerativeModel
import vertexai
import PIL.Image
from IPython.display import Image
from IPython.core.display import HTML
import google.generativeai as genai
import pandas as pd



Caching the list of root modules, please wait!
(This will only be done once - type '%rehashx' to reset cache!)



In [4]:
folder_path = "grad-cam/rois/"
target_objects = ["Sky","Road","Roadmark","Soil","Building","House","Pole","Tree",
                  "Trashcan","Shadow of a car","Car","Vehicle","Truck","Powerline","Sign","Banner",
                  "Roadsign","Fence","Bollard","Person","Text","Skyscraper","Mountain","Body of water",
                  "Nothing","Bush","Vegetation",'People',"Field","Flag","Animal"]
output_csv = "gemini_output.csv"

prompt = f"""
Analyze the provided image and identify the objects present. 
Specifically check if any of the following objects are present: {', '.join(target_objects)}.
If you detect other objects not in the list, please mention them as well.
The output should be ONLY a list in square brackets! Example: 
["car", "tree", "vegetation"]
"""
with open('api_key.txt', 'r') as file:
    API_KEY = file.read().strip()
genai.configure(api_key=API_KEY)


model = genai.GenerativeModel(model_name='gemini-1.5-flash')


# 1) If resume from a previous run, load existing results into a DataFrame:
if os.path.exists(output_csv):
    df_results = pd.read_csv(output_csv)
    print(f"Existing CSV found. Loaded {len(df_results)} entries.")
else:
    # Otherwise, create an empty dataframe with our 2 columns
    df_results = pd.DataFrame(columns=["Image Name", "Analysis Result"])

# Convert "Image Name" column to a set for faster membership checks
already_processed = set(df_results["Image Name"].tolist())

new_file_count = 0

# 2) Loop through the folder and process each image
for filename in os.listdir(folder_path):
    # Skip non-image files (and/or use more extensions if needed)
    if not filename.lower().endswith(('.png', '.jpeg', '.jpg')):
        continue

    # Skip if this file was already processed (optional)
    if filename in already_processed:
        print(f"Skipping {filename} (already processed).")
        continue

    image_path = os.path.join(folder_path, filename)
    print(f"Processing image: {image_path}")

    try:
        image = Image(image_path)
        
        # Use the model to generate text
        response = model.generate_content([prompt, image])
        
        # Extract the generated text from response
        analysis_result = response.text

        if analysis_result:
            df_results.loc[len(df_results)] = [filename, analysis_result]
            print(f"Analysis result for {filename} collected.")
        else:
            df_results.loc[len(df_results)] = [filename, "Failed to analyze"]
            print(f"Failed to analyze {filename}, result collected.")

    except requests.exceptions.RequestException as e:
        print(f"Error making API request: {e}")
        df_results.loc[len(df_results)] = [filename, "Error making API request"]

    except (KeyError, IndexError) as e:
        print(f"Error extracting content from the response: {e}")
        df_results.loc[len(df_results)] = [filename, "Error extracting content"]

    except FileNotFoundError:
        print(f"Error: Image file not found at {image_path}")
        df_results.loc[len(df_results)] = [filename, "File not found"]

    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        df_results.loc[len(df_results)] = [filename, f"Unexpected error: {e}"]

    new_file_count += 1
    # Save partial progress to CSV
    if new_file_count % 10 == 0:
        df_results.to_csv(output_csv, index=False, encoding='utf-8-sig')
        print(f"Saved partial progress after processing {new_file_count} new images.")

print(f"Analysis complete. Final results saved to {output_csv}.")


Existing CSV found. Loaded 10 entries.
Skipping roi_0_0.jpeg (already processed).
Skipping roi_10006_0.jpeg (already processed).
Skipping roi_10009_0.jpeg (already processed).
Skipping roi_10009_1.jpeg (already processed).
Skipping roi_10009_2.jpeg (already processed).
Skipping roi_10009_3.jpeg (already processed).
Skipping roi_10011_0.jpeg (already processed).
Skipping roi_10012_0.jpeg (already processed).
Skipping roi_10012_1.jpeg (already processed).
Skipping roi_10012_2.jpeg (already processed).
Processing image: grad-cam/rois/roi_10012_3.jpeg
Analysis result for roi_10012_3.jpeg collected.
Processing image: grad-cam/rois/roi_10013_0.jpeg
Analysis result for roi_10013_0.jpeg collected.
Processing image: grad-cam/rois/roi_10013_1.jpeg
Analysis result for roi_10013_1.jpeg collected.
Processing image: grad-cam/rois/roi_10013_2.jpeg
Analysis result for roi_10013_2.jpeg collected.
Processing image: grad-cam/rois/roi_10013_3.jpeg
Analysis result for roi_10013_3.jpeg collected.
Processing